In [ ]:
import numpy as np
import pandas as pd
import datetime
#import dcarte
import pytz
#%load_ext autoreload
import pyarrow as pa
import pyarrow.parquet as pq

from random import randrange
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
sleep = pd.read_parquet('sleep_data.parquet', engine='pyarrow')
sleep = sleep.drop('duplicates', axis=1)
ids_list = sleep['ID'].unique()

In [ ]:
len(ids_list)

In [ ]:
def process_sleep_mat(patient_id, df):
    df = df[df['ID']==patient_id].reset_index(drop=True)

    df.index = pd.DatetimeIndex(df['start_date_tz'])
    df = df.drop(columns=['start_date_tz'])
    df = df.resample('1T').asfreq()#.ffill()
    df = df.rename_axis('datetime').reset_index()
    df['value'] = np.where(df['state'].isna(), 'bed_out', 'bed_in')
    df['value_diff'] = df['value']!=df['value'].shift(1)
    df['ID'] = patient_id
    return df[df['value_diff']==True][['ID','datetime','value']].reset_index(drop=True)

In [ ]:
def start_table():
    return pd.DataFrame(columns=['Participant ID','date','cluster','datetime','end_datetime','length','classification'])
def append_row(df, row):
    return pd.concat([ df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)
def get_date(df):
        df['hour'] = df['datetime'].dt.hour
        df['date'] = df['datetime'].dt.date
        df['delta'] = pd.Timedelta(days=0)
        df.loc[df['hour']<7, 'delta'] = pd.Timedelta(days=1)
        df['date'] = df['date']-df['delta']
        return df[['date','datetime', 'end_datetime','length']]

In [ ]:
table=start_table()
for i in ids_list:
    if i!='Xv41A':
        i_df = process_sleep_mat(i, sleep)
        i_df['duration_in_minutes'] = (i_df['datetime'].shift(-1)-i_df['datetime']).dt.total_seconds()/60
        i_df['datetime'] = i_df['datetime'].dt.tz_convert(pytz.timezone('Europe/London'))
        i_df['end_datetime'] = i_df['datetime'].shift(-1)
        i_df['duration_in_minutes'] = (i_df['end_datetime']-i_df['datetime']).dt.total_seconds()/60
        i_df = i_df[i_df['value']=='bed_in']
        i_df = i_df.assign(gap  = (i_df['datetime'] - i_df['end_datetime'].shift()).dt.total_seconds()/60 )
        i_df = i_df.assign(cluster  = ((i_df['gap'] > 60).cumsum()))
        group = i_df.groupby(['cluster']).aggregate({'datetime':min,'end_datetime':max})
        group['length'] = (group['end_datetime']-group['datetime']).dt.total_seconds()/60
        group = get_date(group)
        group['Participant ID'] = i
        group['classification'] = 'diurnal'
        group.loc[(group['datetime'].dt.hour>=18)|(group['datetime'].dt.hour<7), 'classification'] = 'nocturnal'


        table  = pd.concat([ table, group.reset_index()]).reset_index(drop=True)
        df2 = pa.Table.from_pandas(table)
        pq.write_table(df2, 'clusters_hour_gap.parquet', compression='BROTLI')

table

C:\Users\ir0011\AppData\Local\Temp\ipykernel_13324\633938861.py:10: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['date'] = df['date']-df['delta']
C:\Users\ir0011\AppData\Local\Temp\ipykernel_13324\633938861.py:10: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['date'] = df['date']-df['delta']
C:\Users\ir0011\AppData\Local\Temp\ipykernel_13324\633938861.py:10: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['date'] = df['date']-df['delta']
C:\Users\ir0011\AppData\Local\Temp\ipykernel_13324\633938861.py:10: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['date'] = df['date']-df['delta']
C:\Users\ir0011\AppData\Local\Temp\ipykernel_13324\633938861.py:10: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  df['date'] = df['date']-df['delta']
C:\Users\i

,Participant ID,date,cluster,datetime,end_datetime,length,classification
0,Mhy2u,2019-03-31,0,2019-04-01 01:00:00+01:00,2019-04-01 08:40:00+01:00,460.0,nocturnal
1,Mhy2u,2019-04-01,1,2019-04-01 22:05:00+01:00,2019-04-02 07:37:00+01:00,572.0,nocturnal
2,Mhy2u,2019-04-02,2,2019-04-02 22:56:00+01:00,2019-04-03 07:50:00+01:00,534.0,nocturnal
3,Mhy2u,2019-04-03,3,2019-04-03 22:43:00+01:00,2019-04-04 07:19:00+01:00,516.0,nocturnal
4,Mhy2u,2019-04-04,4,2019-04-04 18:30:00+01:00,2019-04-04 20:41:00+01:00,131.0,nocturnal
...,...,...,...,...,...,...,...
61623,QbsYB,2023-06-15,35,2023-06-15 23:32:00+01:00,2023-06-16 07:45:00+01:00,493.0,nocturnal
61624,QbsYB,2023-06-16,36,2023-06-17 00:30:00+01:00,2023-06-17 07:59:00+01:00,449.0,nocturnal
61625,QbsYB,2023-06-17,37,2023-06-18 00:34:00+01:00,2023-06-18 07:32:00+01:00,418.0,nocturnal
61626,QbsYB,2023-06-18,38,2023-06-19 00:20:00+01:00,2023-06-19 07:26:00+01:00,426.0,nocturnal


table = pd.read_parquet('clusters_hour_gap.parquet', engine='pyarrow')

In [ ]:
table['time'] = table['datetime'].dt.hour

sns.histplot(data=table, x='time',hue='classification',binwidth=1)

In [ ]:
table.sort_values(by=['length'])

In [ ]:
table[(table['time']<7)&(table['time']>5)]